In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2

# %load init.ipy
import os, sys, logging, datetime, warnings, shutil
from importlib import reload

import numpy as np
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
from nose import tools

import kalepy as kale
import kalepy.utils
import kalepy.plot
import kalepy.sample

# The `nbshow` command runs `plt.show()` in interactive jupyter notebooks, but closes
#   figures when run from the command-line (notebooks are converted to scripts and run as tests)
from kalepy.plot import nbshow

import warnings
# warnings.simplefilter("error")   # WARNING: this is breaking jupyter at the moment (2021-02-14)

In [ ]:
import zcode.plot as zplot
import zcode.math as zmath

In [ ]:
def draw_test(xx, yy, data):
    fig, axes = plt.subplots(figsize=[12, 4], ncols=2)

    for ii, ax in enumerate(axes):
        interp = bool(ii)
        vals = kale.sample.sample_grid([xx, yy], data, 2000, interpolate=interp)
        ax.set_title(f"{interp=}")
        ax.scatter(*vals, color='pink', s=10, zorder=20, alpha=0.3)
        ax.hist2d(*vals, bins=10)

    return fig

In [ ]:
xx = [0.0, 1.0, 2.0]
yy = [100.0, 200.0, 300.0]

edge = [
    [0.0, 10.0, 0.0],
    [10.0, 20.0, 10.0],
    [0.0, 10.0, 0.0]
]

draw_test(xx, yy, edge)

plt.show()

In [ ]:
xx = [0.0, 1.0, 2.0]
yy = [100.0, 200.0, 300.0]

edge = [
    [10.0, 0.0, 10.0],
    [ 0.0, 0.0,  0.0],
    [10.0, 0.0, 10.0]
]

draw_test(xx, yy, edge)
plt.show()

In [ ]:
xx = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
yy = [100.0, 200.0, 300.0]

edge = [
    [10.0, 0.0, 10.0],
    [10.0, 0.0, 10.0],
    [10.0, 0.0, 10.0],
    [10.0, 0.0, 10.0],
    [10.0, 0.0, 10.0],
    [10.0, 0.0, 10.0]
]

draw_test(xx, yy, edge)
plt.show()

In [ ]:
def func(x, y, z):
    rv = (x + y - 2*z)**2 + x**2 + y**2 + z**2
    rv = 3 * np.exp(-rv / 2.0)
    return rv

xx = np.linspace(-4, 4, 200)
yy = np.linspace(-4, 4, 300)
zz = np.linspace(-1, 1, 5)

data = func(xx[:, np.newaxis, np.newaxis], yy[np.newaxis, :, np.newaxis], zz[np.newaxis, np.newaxis, :]) 

smap = zplot.smap(data, scale='linear')

for ii, _ in enumerate(zz):
    fig, ax = plt.subplots()
    aa = data[:, :, ii]
    plt.pcolormesh(xx, yy, aa.T, cmap=smap.cmap, norm=smap.norm)
    
plt.show()
    

In [ ]:
vals = kale.sample.sample_grid([xx, yy, zz], data, 2000)

In [ ]:
smap = zplot.smap(data_edge, scale='lin')

keys = zmath.argnearest(zz, vals[2])
for ii in range(zz.size):
    idx = (keys == ii)
        
    fig, ax = plt.subplots()
    plt.pcolormesh(xx, yy, data[:, :, ii].T, cmap=smap.cmap, norm=smap.norm)
    uu = vals[0][idx]
    vv = vals[1][idx]
    plt.scatter(uu, vv, facecolor='pink', edgecolor='0.25', alpha=0.7, s=20, zorder=100)

plt.show()

In [ ]:
kale.corner(kale.utils._random_data_3d_01())